In [ ]:
import os 
import gradio as gr
from dotenv import load_dotenv
from openai import OpenAI

d:\Agnetic AI\9am\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
load_dotenv(override=True)
groq_api_key = os.getenv("GROQ_API_KEY")
groq_url = "https://api.groq.com/openai/v1"
groq = OpenAI(
    base_url=groq_url,
    api_key= groq_api_key
)

In [3]:
model = "openai/gpt-oss-120b"
client = groq

In [11]:
system_prompt = """
You are an English Grammar Fixer.
Your task is to correct grammar, spelling, punctuation, capitalization, and sentence structure in the user’s text without changing the original meaning or tone.
General Rules:
Preserve the user’s voice, intent, and level of formality.
Do not add new ideas or remove content unless necessary for clarity.
Keep wording as close to the original as possible.
Fix awkward phrasing only when it clearly improves readability.
If multiple corrections are possible, choose the most natural and commonly accepted form.
If the input is already correct, return it unchanged.
Output Rules (Always Explain):
Always show the corrected text first.
Then provide a clear, concise explanation of all changes made.
Explain what was changed and why (grammar, punctuation, clarity, or usage).
Focus on grammar rules and correctness, not personal style preferences.
Keep explanations brief, simple, and easy to understand.
If no changes were needed, state that clearly and explain why the text is already correct.
"""

In [7]:
def chat(message, history):
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": message}
    ]
    stream = client.chat.completions.create(
        model = model,
        messages = messages,
        stream = True
    )
    response = ""
    for chunk in stream:
        if not getattr(chunk, "choices", None):
            continue

        choice = chunk.choices[0]
        delta = getattr(choice, "delta", None)
        if not delta:
            continue

        text = getattr(delta, "content", None)
        if not text:
            continue
        
        response += text
        yield response


In [13]:
gr.ChatInterface(fn=chat, type="messages").launch(inbrowser=True)

* Running on local URL:  http://127.0.0.1:7864
* To create a public link, set `share=True` in `launch()`.
